In [2]:
import numpy as np
import pandas as pd
import networkx as nx
train_data = pd.read_csv('data_train_edge.csv')
test_data = pd.read_csv('predict.csv')
ans_data = pd.read_csv('ans500_ground_truth.csv')

In [15]:
print(ans_data['ans'])
print(train_data[0:10])

0      0
1      0
2      1
3      1
4      1
      ..
495    1
496    0
497    0
498    0
499    0
Name: ans, Length: 500, dtype: int64
   node1  node2        JF
0    287     68  0.196078
1     63    552  0.256410
2    189     20  0.166667
3    380    376  0.300000
4    370    443  0.425926
5     87    427  0.147651
6    212    411  0.122905
7    407    525  0.217391
8     28    128  0.290698
9    133    568  0.088235


In [4]:
G = nx.from_pandas_edgelist(train_data, 'node1', 'node2')

In [5]:
test_jc = []
for i in range(len(test_data)):
    try:
        preds = nx.jaccard_coefficient(G, [(test_data['node1'][i], test_data['node2'][i])])
        for u, v, p in preds:
            test_jc.append(p)
    except:
        test_jc.append(0)

In [6]:
print(test_jc[1])
test_data['JF'] = test_jc
test_data.shape

0.044444444444444446


(10200, 3)

In [7]:
train_jc = []
for i in range(len(train_data)):
    try:
        preds = nx.jaccard_coefficient(G, [(train_data['node1'][i], train_data['node2'][i])])
        for u, v, p in preds:
            train_jc.append(p)
    except:
        train_jc.append(0)


In [8]:
trainX= pd.DataFrame(train_data)
trainX['node1']

0        287
1         63
2        189
3        380
4        370
        ... 
20452     59
20453    210
20454    547
20455     57
20456    339
Name: node1, Length: 20457, dtype: int64

In [11]:
trainX['JF'] = train_jc 
print(trainX[:])

       node1  node2        JF
0        287     68  0.196078
1         63    552  0.256410
2        189     20  0.166667
3        380    376  0.300000
4        370    443  0.425926
...      ...    ...       ...
20452     59     59  0.978723
20453    210    420  0.175141
20454    547    417  0.269841
20455     57    924  0.075758
20456    339     10  0.056818

[20457 rows x 3 columns]


In [16]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(trainX[0:500],ans_data['ans'])

RandomForestClassifier()

In [19]:
ans = pd.DataFrame()
ans['ans'] = rf.predict(trainX[500:])



In [40]:
ans_final = pd.DataFrame()
#print(ans['ans'])
#print(ans_data['ans'])

ans_final['ans'] = pd.concat([ans_data['ans'],ans['ans']])
print(ans_final)

       ans
0        0
1        0
2        1
3        1
4        1
...    ...
19952    1
19953    0
19954    0
19955    0
19956    1

[20457 rows x 1 columns]


In [41]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(trainX,ans_final['ans'])

RandomForestClassifier()

In [42]:
predict_ans = pd.DataFrame()
predict_ans['ans'] = rf.predict(test_data)

In [43]:
print(predict_ans['ans'])

0        0
1        0
2        1
3        0
4        1
        ..
10195    0
10196    1
10197    1
10198    0
10199    1
Name: ans, Length: 10200, dtype: int64


In [44]:
ans_predict = pd.DataFrame()
ans_predict['predict_nodepair_id'] = [_ for _ in range(10200)]
ans_predict['ans'] = predict_ans['ans']
ans_predict.to_csv('rf_predict.csv', index= False)